<a href="https://colab.research.google.com/github/walexand3r/4YP/blob/master/CIFAR10/ML/Preliminary%20Testing/v1.2(HSV)_CNN_for_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reproducing Jack Pegg's work, using CNN on CIFAR-10 dataset on HSV Values. 

Initial testing of HSV/RGB only

Model Version 1.2

Added: 




In [0]:

# This block imports relevant packages and sets intial seed to ensure reproducability
# Seed value
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os, datetime
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
%tensorflow_version 2.x
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: tf.random.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
import tensorflow.keras as keras
# from tensorflow.keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Dropout, Flatten,  Reshape, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Softmax
from tensorflow import math
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2hsv, hsv2rgb
from matplotlib import pyplot as plt


%load_ext tensorboard
import pandas as pd

In [0]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive/')

In [0]:
# Setting paths
# Import data path
folder = 'HSV'
path = '/content/drive/My Drive/4th Year/4YP/CIFAR10/Processed Datasets/'
pathTrain  = path[:74] + folder + '/train_hsv.csv'
pathTest = path[:74] + folder + '/test_hsv.csv'

# Sets results rath
folder = 'HSV'
savePath = '/content/drive/My Drive/4th Year/4YP/CIFAR-10/Results/'

In [0]:
# Load CIFAR_10 data
train = pd.read_csv(pathTrain,header=None)
test = pd.read_csv(pathTest,header=None)

(n_train,row_length) = np.shape(train)
pixels = (row_length - 1) / 3
n_test = np.size(test,0)
dim = int(np.sqrt(pixels))
num_classes = 10

# Getting data in right form
# input image dimensions
img_rows, img_cols = dim, dim

# Extracting data
test_labels = test.iloc[:,0]
test_values = test.iloc[:,1:] 
train_labels = train.iloc[:,0]
train_values = train.iloc[:,1:] 
x_train = train_values.values.reshape(n_train,img_rows,img_cols,3)
x_test = test_values.values.reshape(n_test,img_rows,img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

In [0]:
batch_size = 64
epochs = 50

inputs = keras.Input(shape=(32, 32, 3))
x = Conv2D(64, kernel_size = (3,3), padding = 'same')(inputs)
x = LeakyReLU(alpha = 0.3)(x)

x = Conv2D(64, kernel_size=(3, 3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(128, kernel_size=(3, 3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)

x = Conv2D(128, kernel_size=(3, 3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(256, kernel_size=(3, 3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)

x = Conv2D(256, kernel_size=(1, 1), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(10, kernel_size=(3, 3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.3)(x)
x = tf.reduce_mean(x,(1,2))
outputs = Softmax()(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='cifar_model')

model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.98),
              metrics=['accuracy'])

#model.summary()
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.02)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [0]:
!kill 725
%tensorboard --logdir logs  # start Tensorboard
